### SQL Views and Subselect

In [1]:
import pandas as pd 
import sqlite3

#### simple subselect

```SQL
-- world.db

CREATE TABLE t ( a TEXT, b TEXT );
INSERT INTO t VALUES ( 'NY0123', 'US4567' );
INSERT INTO t VALUES ( 'AZ9437', 'GB1234' );
INSERT INTO t VALUES ( 'CA1279', 'FR5678' );
SELECT * FROM t;

SELECT SUBSTR(a, 1, 2) AS State, SUBSTR(a, 3) AS SCode, 
  SUBSTR(b, 1, 2) AS Country, SUBSTR(b, 3) AS CCode FROM t;

SELECT co.Name, ss.CCode FROM (
    SELECT SUBSTR(a, 1, 2) AS State, SUBSTR(a, 3) AS SCode,
      SUBSTR(b, 1, 2) AS Country, SUBSTR(b, 3) AS CCode FROM t
  ) AS ss
  JOIN Country AS co
    ON co.Code2 = ss.Country
;

DROP TABLE t;
```

#### searching within a result set

```SQL
-- album.db

SELECT DISTINCT album_id FROM track WHERE duration <= 90;

SELECT * FROM album
  WHERE id IN (SELECT DISTINCT album_id FROM track WHERE duration <= 90)
;

SELECT a.title AS album, a.artist, t.track_number AS seq, t.title, t.duration AS secs
  FROM album AS a
  JOIN track AS t
    ON t.album_id = a.id
  WHERE a.id IN (SELECT DISTINCT album_id FROM track WHERE duration <= 90)
  ORDER BY a.title, t.track_number
;

SELECT a.title AS album, a.artist, t.track_number AS seq, t.title, t.duration AS secs
  FROM album AS a
  JOIN (
    SELECT DISTINCT album_id, track_number, duration, title
      FROM track
      WHERE duration <= 90
  ) AS t
    ON t.album_id = a.id
  ORDER BY a.title, t.track_number
;
```

In [4]:

con = sqlite3.connect("data/album.db")
pd.read_sql_query("SELECT DISTINCT album_id FROM track WHERE duration <= 90;", con)

,album_id
0,11
1,13


#### Creating a view

```SQL
-- album.db

SELECT id, album_id, title, track_number, 
  duration / 60 AS m, duration % 60 AS s FROM track;

CREATE VIEW trackView AS
  SELECT id, album_id, title, track_number, 
    duration / 60 AS m, duration % 60 AS s FROM track;
SELECT * FROM trackView;

SELECT a.title AS album, a.artist, t.track_number AS seq, t.title, t.m, t.s
  FROM album AS a
  JOIN trackView AS t
    ON t.album_id = a.id
  ORDER BY a.title, t.track_number
;

DROP VIEW IF EXISTS trackView;
```

In [10]:
pd.read_sql_query('''
SELECT id, album_id, title, track_number, 
duration / 60 AS m, duration % 60 AS s FROM track;
''', con)

,id,album_id,title,track_number,m,s
0,1,1,Bright Lights Big City,1,5,20
1,2,1,Night Life,2,5,44
2,3,1,Basin Street Blues,5,4,56
3,4,1,Caldonia,3,3,25
4,5,1,Stardust,4,5,8
...,...,...,...,...,...,...
58,66,18,Freddy Freeloader,2,9,49
59,67,18,Blue in Green,3,5,38
60,68,18,All Blues,4,11,36
61,69,18,Flamenco Sketches,5,9,26


#### Joined view

```SQL 
-- album.db

SELECT a.artist AS artist,
    a.title AS album,
    t.title AS track,
    t.track_number AS trackno,
    t.duration / 60 AS m,
    t.duration % 60 AS s
  FROM track AS t
    JOIN album AS a
      ON a.id = t.album_id
;

CREATE VIEW joinedAlbum AS
  SELECT a.artist AS artist,
      a.title AS album,
      t.title AS track,
      t.track_number AS trackno,
      t.duration / 60 AS m,
      t.duration % 60 AS s
    FROM track AS t
    JOIN album AS a
      ON a.id = t.album_id
;

SELECT * FROM joinedAlbum;
SELECT * FROM joinedAlbum WHERE artist = 'Jimi Hendrix';

SELECT artist, album, track, trackno, 
   m || ':' || substr('00' || s, -2, 2) AS duration
    FROM joinedAlbum;

DROP VIEW IF EXISTS joinedAlbum;
```

In [15]:
pd.read_sql_query('''
SELECT a.artist AS artist,
    a.title AS album,
    t.title AS track,
    t.track_number AS trackno,
    t.duration / 60 AS m,
    t.duration % 60 AS s
  FROM track AS t
    JOIN album AS a
      ON a.id = t.album_id; ''', con).head(10)

,artist,album,track,trackno,m,s
0,Willie Nelson and Wynton Marsalis,Two Men with the Blues,Bright Lights Big City,1,5,20
1,Willie Nelson and Wynton Marsalis,Two Men with the Blues,Night Life,2,5,44
2,Willie Nelson and Wynton Marsalis,Two Men with the Blues,Basin Street Blues,5,4,56
3,Willie Nelson and Wynton Marsalis,Two Men with the Blues,Caldonia,3,3,25
4,Willie Nelson and Wynton Marsalis,Two Men with the Blues,Stardust,4,5,8
5,Willie Nelson and Wynton Marsalis,Two Men with the Blues,Georgia On My Mind,6,4,40
6,Willie Nelson and Wynton Marsalis,Two Men with the Blues,Rainy Day Blues,7,5,43
7,Willie Nelson and Wynton Marsalis,Two Men with the Blues,My Bucket's Got A Hole In It,8,4,56
8,Willie Nelson and Wynton Marsalis,Two Men with the Blues,Ain't Nobody's Business,9,7,27
9,Willie Nelson and Wynton Marsalis,Two Men with the Blues,That's All,10,6,8
